# ***Q1***

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

image = torch.rand(6,6)
print("image=", image)
#Add a new dimension along 0th dimension
#i.e. (6,6) becomes (1,6,6). This is because
#pytorch expects the input to conv2D as 4d tensor
image = image.unsqueeze(dim=0)
print("image.shape=", image.shape)
image = image.unsqueeze(dim=0)
print("image.shape=", image.shape)
print("image=", image)
kernel = torch.ones(3,3)
#kernel = torch.rand(3,3)
print("kernel=", kernel)
kernel = kernel.unsqueeze(dim=0)
kernel = kernel.unsqueeze(dim=0)
#Perform the convolution
outimage = F.conv2d(image, kernel, stride=1, padding=0)
conv_layer = torch.nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, stride=1, padding=0)

# Apply convolution
output_image = conv_layer(image)

# Print output dimensions and total parameters
print("Output dimensions with stride=1 and no padding:", output_image.shape)
print("Total parameters:", sum(p.numel() for p in conv_layer.parameters()))
print("outimage=", outimage)

# ***Q2***

In [ ]:
conv_layer = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)

# Apply convolution
output_image = conv_layer(image)

# Print output dimensions
print("Output dimensions using nn.Conv2d:", output_image.shape)

# ***Q3***

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.metrics import confusion_matrix

# Step 1: Import necessary libraries

# Step 2: Load and normalize the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Step 3: Define the CNN architecture
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 4 * 4, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Step 4: Set hyperparameters and define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Step 5: Train the model
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d Loss: %.3f' % (epoch + 1, running_loss / (i + 1)))
print('Finished Training')

# Step 6: Test the model and display the confusion matrix
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

conf_mat = confusion_matrix(labels.numpy(), predicted.numpy())
print("Confusion Matrix:\n", conf_mat)

# Step 7: Verify the number of learnable parameters in the model
num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Number of learnable parameters: ", num_params)

# ***Q4***

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Define the CNN architecture with fewer parameters
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 4, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(4, 12, 5)
        self.fc1 = torch.nn.Linear(12 * 4 * 4, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Set hyperparameters and define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Initialize lists to store training loss, validation accuracy, and number of parameters
train_losses = []
val_accuracies = []
num_params_list = []

# Train the model and record the metrics at every epoch
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    # Compute validation accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    acc = 100.*correct/total
    val_accuracies.append(acc)
    
    # Record the number of parameters in the model
    num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    num_params_list.append(num_params)
    
    # Print metrics
    print('Epoch %d Loss: %.3f Val Acc: %.2f%% Params: %d' % (epoch + 1, running_loss / (i + 1), acc, num_params))
    
# Plot percentage drop in parameters vs accuracy
param_drop = 100 - np.array(num_params_list[1:])/np.array(num_params_list[:-1])*100
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(range(1,len(val_accuracies)+1), val_accuracies)
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylim([70,100])

plt.subplot(1, 2, 2)
plt.plot(range(1,len(param_drop)), param_drop)
plt.title('Percentage Drop in Parameters')
plt.xlabel('Epoch')
plt.ylabel('Parameter Drop (%)')
plt.show()

NameError: name 'trainloader' is not defined